### Import Libraries

In [ ]:
import numpy as np
import time
import random
import matplotlib.pyplot as plot
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle

### Function downloaded from the link provided

In [ ]:
class Function1 :
        
    def cnn_function(self, paramters):
        batch_size, alpha, beta1, beta2, epochs = paramters
        batch_size = int(batch_size)
        epochs = int(epochs)
        (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
        n = 5000
        x_train = x_train[1:n]
        y_train = y_train[1:n]
        num_classes = 10
        input_shape = (32, 32, 3)
        x_train = x_train.astype("float32") / 255
        x_test = x_test.astype("float32") / 255
        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)
        model = keras.Sequential()
        model.add(Conv2D(16, (3, 3), padding='same',
        input_shape=x_train.shape[1:], activation='relu'))
        model.add(Conv2D(16, (3, 3), strides=(2, 2), padding='same', activation='relu'))
        model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu'))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(num_classes, activation='softmax',
        kernel_regularizer=regularizers.l1(0.0001)))
        optimizer = Adam(learning_rate=alpha, beta_1=beta1, beta_2=beta2)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer,
        metrics=["accuracy"])
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
        validation_split=0.1, verbose=0)
        y_preds = model.predict(x_test)
        loss = CategoricalCrossentropy()
        return loss(y_test, y_preds).numpy()
    

###  Implement the Global Random Seach and Global Population Search 

In [ ]:
class Global_Random_Population_Search : 
    
    def execute_global_random_search(self, function_object, num_parameters = 2, lower_bound = [0,0], upper_bound = [5,5], num_samples = 100) : 
        
        cost_iteration = []
        time_iteration = []
        optimised_cost = float('inf')
        optimised_params = None
        params_iteration = []
        
        start_time = time.time() * 1000
        
        for i in range(num_samples) : 
            print(f'num_samples:{i}')
            parameter = [random.uniform(lower_bound[j], upper_bound[j]) for j in range(num_parameters)]
            current_cost = function_object.cnn_function(parameter)
            
            if current_cost < optimised_cost : 
                optimised_cost = current_cost
                optimised_params = parameter
            
            params_iteration.append(optimised_params)
            cost_iteration.append(optimised_cost)
            time_iteration.append((time.time() * 1000) - start_time)
            
        return cost_iteration, time_iteration, params_iteration
    
    
    
     def execute_global_population_search(self, function_object, num_parameters = 2, lower_bound = [0,0], upper_bound = [5,5], num_samples = 100, M_best_params = 10, num_iterations = 200) : 
        
        cost_iteration = []
        time_iteration = []
        optimised_cost = float('inf')
        start_time = time.time() * 1000
        optimised_params = None
        params_iteration = []
        
        n_sample_cost = []
        
        for i in range(num_samples) : 
            
            parameter_x, parameter_y = [random.uniform(lower_bound[j], upper_bound[j]) for j in range(num_parameters)]
            current_cost = function_object.get_quadratic_equation_value(parameter_x, parameter_y)
            
            if current_cost < optimised_cost : 
                optimised_cost = current_cost
                optimised_params = [parameter_x, parameter_y]
            
            n_sample_cost.append(optimised_cost)
            cost_iteration.append(optimised_cost)
            params_iteration.append([parameter_x, parameter_y])
            time_iteration.append((time.time() * 1000) - start_time)
            
        for _ in range(num_iterations) : 
            print(f'num_iterations: {_}')
            best_cost = sorted(n_sample_cost)[:M_best_params]
            best_params = [params_iteration[n_sample_cost.index(cost)] for cost in n_sample_cost]
            
            best_cost_iteration = []
            best_param_iteration = []
            best_time_iteration = []
            
            for best_params in best_params : 
                for _ in range(num_samples) : 
                    sample_parameter_x, sample_parameter_y = [random.uniform(best_params[j] - (upper_bound[j] - lower_bound[j]) * 0.1, best_params[j] + (upper_bound[j] - lower_bound[j]) * 0.1) for j in range(num_parameters)]
                    sample_cost = function_object.get_quadratic_equation_value(sample_parameter_x, sample_parameter_y)

                    if sample_cost < optimised_cost : 
                        optimised_cost = sample_cost
                        optimised_params = [sample_parameter_x, sample_parameter_y]

                    best_cost_iteration.append(optimised_cost)
                    cost_iteration.append(optimised_cost)
                    best_param_iteration.append(optimised_params)
                    best_time_iteration.append(time.time() - start_time)
            
            params_iteration.extend(best_param_iteration)
            n_sample_cost = best_cost_iteration
            time_iteration.extend(best_time_iteration)
            
        return cost_iteration, time_iteration, params_iteration




#### Executing Global Random Search for CNN Function

In [ ]:
function1 = Function1()
grs = Global_Random_Population_Search()

num_parameters = 5
lower_bound = [16, 0.0001,0.25,0.9,10]
upper_bound = [64,0.01,0.9,0.999,20]
num_iterations = 100

grs_cost_iteration_func1, grs_time_iteration_func1, grs_param_iteration_func1 = grs.execute_global_random_search(function_object = function1, num_parameters = num_parameters, lower_bound = lower_bound, upper_bound = upper_bound, num_samples = num_iterations)

#### Executing Global Population Search for CNN Function

In [ ]:
function1 = Function1()
grs = Global_Random_Population_Search()

num_parameters = 2
lower_bound = [0, 0]
upper_bound = [10, 10]
num_samples = 200
M_best_params = 10
num_iterations = 200

gps_cost_iteration_func1, gps_time_iteration_func1, gps_param_iteration_func1 = grs.execute_global_population_search(function_object = function1, num_parameters = num_parameters, lower_bound = lower_bound, upper_bound = upper_bound,  num_samples = 10, M_best_params = 2, num_iterations = 5)

#### Plots for CNN Function

In [ ]:
plot.figure(figsize=(11, 6), dpi=150)
plot.semilogy(range(num_iterations), grs_cost_iteration_func1, color = 'cyan', label = 'Global Random Search')
plot.semilogy(range(num_iterations), gps_cost_iteration_func1, color = 'yellow', label = 'Global Population Search')
plot.xlabel('#Iteration')
plot.ylabel('Cost')
plot.title('Plot of Function Cost vs. Iteration for GRS vs. GPS for CNN Function') 
plot.yscale("log")
plot.legend()
plot.show()